In [1]:
#Уберем предупреждения, чтобы они не загромождали вывод
import warnings 
warnings.filterwarnings('ignore')

In [2]:
client_id = 'bcf40466-3e35-4583-a25b-a73219d9f796'
secret = '35709e53-1531-40f5-8fe2-bfb739a190e8'
auth = 'YmNmNDA0NjYtM2UzNS00NTgzLWEyNWItYTczMjE5ZDlmNzk2OjM1NzA5ZTUzLTE1MzEtNDBmNS04ZmUyLWJmYjczOWExOTBlOA=='

In [6]:
# auth = 'ODQ4ODM5MWQtYWViZi00ZGU1LWEzODItMjA5NzRkMDE3Y2VmOmVhOWU3YjRlLTQ2MGEtNDE4Yy1iNZI0LWMxM2M4YmZlNDc3MA=='

In [3]:
import base64
credentials = f"{client_id}:{secret}"
encoded_credentials = base64.b64encode(credentials.encode('utf-8')).decode('utf-8')

In [4]:
auth == encoded_credentials

True

In [4]:
import requests
import uuid

In [13]:
def get_token(auth_token, scope = 'GIGACHAT_API_PERS'):
    """
    Выполняет POST-запрос к эндпоинту, который выдает токен.

    Параметры:
    - auth_token (str): токен авторизации, необходимый для запроса.
    - область (str): область действия запроса API. По умолчанию - "GIGACHAT_API_PERS" (физическое лицо, иначе юридическое).

    Возвращает:
    - ответ API, где токен и срок его "годности".
    """

    # создадим идентификатор UUID (36 знаков)
    rq_uid = str(uuid.uuid4())

    # API URL
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    # Заголовки
    headers = {
      'Content-Type': 'application/x-www-form-urlencoded',
      'Accept': 'application/json',
      'RqUID': rq_uid,
      'Authorization': f'Basic {auth_token}'
    }

    # Тело запроса
    payload = {"scope" : scope}

    try:
        # делаем POST запрос с отключенной SSL верификацией
        # (можно скачать сертификаты Минцифры, тогда отлюкчать проверку не надо)
        response = requests.post(url, headers = headers, data = payload, verify = False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return -1

In [14]:
response = get_token(auth)
if response != -1:
    print(response.text)
    giga_token = response.json()["access_token"]

Ошибка: HTTPSConnectionPool(host='ngw.devices.sberbank.ru', port=9443): Max retries exceeded with url: /api/v2/oauth (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))


Получение перечня моделей

In [8]:
import requests

url = "https://gigachat.devices.sberbank.ru/api/v1/models"

payload =  {}
headers = {
  'Accept': 'application/json',
  'Authorization': f'Bearer {giga_token}'
}

response = requests.request("GET", url, headers=headers, data=payload, verify = False) # verify = False - непроверенный сертификат шифрования

print(response.text)

{"object":"list","data":[{"id":"GigaChat","object":"model","owned_by":"salutedevices"},{"id":"GigaChat-Plus","object":"model","owned_by":"salutedevices"},{"id":"GigaChat-Pro","object":"model","owned_by":"salutedevices"},{"id":"GigaChat-Pro-preview","object":"model","owned_by":"salutedevices"}]}


In [24]:
import requests
import json

def get_chat_completion(auth_token, user_message):
    """
    Отправляет POST-запрос к API чата для получения ответа от модели GigaChat.

    Параметры:
    - auth_token (str): Токен для авторизации в API
    - user_message (str): Сообщение от пользователя, для которого нужно получить ответ.

    Возвращает:
    - str: Ответ от API в виде текстовой строки.
    
    """

    # URL API, к которому мы обращаемся
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    # Подготовка данных запроса в формате JSON
    payload = json.dumps({
          "model": "GigaChat", # Используемая модель
          "messages": [
            {
              "role": "user", # Роль отправителя (пользователь)
              "content": user_message # Содержание сообщения
            }
          ],
          "temperature": 1, # Генерация температуры (насколько случайным будет ответ при генерации ответа. Если 0, то будет выбираться самый надежный результат (всегда один и тот же ответ))
          "top_p": 0.1, # Параметр top_p для контроля разнообразия ответов (также за разнообразие)
          "n": 1, # Количество возвращаемых ответов
          "stream": False, # Потоквая ли передача ответов (отправляется ли каждый токен в ответ пользователю или сразу весь ответ)
          "max_tokens": 512, # Максимальное количество токенов в ответе
          "repetition_penalty": 1, # Штраф за повторения (можно ставить побольше, чтобы не повторялись в больших текстах)
          "update_interval": 0 # Интервал обновления (для потоковой передачи)
        })

    # Заголовки запроса
    headers = {
      'Content-Type': 'application/json', # Тип содержимого - JSON
      'Accept': 'application/json', # Принимаем ответ в формате JSON
      'Authorization': f'Bearer {auth_token}' # Токен авторизации
    }

    try:
        # response = requests.post("POST", url, headers=headers, data=payload, verify=False)
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        # Обработка исключения в случае ошибки запроса
        print(f"Произошла ошибка: {str(e)}")
        return -1

In [25]:
answer = get_chat_completion(giga_token, "Как на Пайтоне сделать GET запрос")

In [26]:
from IPython.display import display, Markdown
display(Markdown(answer.json()['choices'][0]['message']['content']))

Вы можете использовать модуль `requests` для отправки GET-запросов на Python. Вот базовый пример:

```python
import requests

response = requests.get('http://example.com')

```

В этом примере мы отправляем GET-запрос на URL 'http://example.com' и сохраняем ответ в переменной `response`.

Обратите внимание, что этот код не обрабатывает возможные ошибки или исключения, которые могут возникнуть при отправке запроса. Вам может потребоваться добавить дополнительный код для обработки этих ситуаций.

Создадим диалог

In [37]:
import requests
import json

def get_chat_completion(auth_token, user_message, conversation_history):
    """
    Отправляет POST-запрос к API чата для получения ответа от модели GigaChat.

    Параметры:
    - auth_token (str): Токен для авторизации в API
    - user_message (str): Сообщение от пользователя, для которого нужно получить ответ.
    - conversation_history (list): История диалога в виде списка сообщений (опционально).

    Возвращает:
    - response (requests.esponse): Ответ от API.
    - conversation_history (list): Обновленная история диалога.
    
    """

    # URL API, к которому мы обращаемся
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    # Если история диалога не предоставлена, инициализируем пустыс списком
    if len(conversation_history) == 0:
        conversation_history = []
    else:
        user_message = "Теперь представь что ты гопник из 90х и ответь на другой вопрос пользователя: " + user_message
        print(user_message)

    conversation_history.append({
        "role": "user",
        "content": user_message
    })

    # Подготовка данных запроса в формате JSON
    payload = json.dumps({
          "model": "GigaChat", # Используемая модель
          # ниже список сообщений который и будет каждый раз перезаписываться
          "messages": conversation_history,
          "temperature": 1, # Генерация температуры (насколько случайным будет ответ при генерации ответа. Если 0, то будет выбираться самый надежный результат (всегда один и тот же ответ))
          "top_p": 0.1, # Параметр top_p для контроля разнообразия ответов (также за разнообразие)
          "n": 1, # Количество возвращаемых ответов
          "stream": False, # Потоквая ли передача ответов (отправляется ли каждый токен в ответ пользователю или сразу весь ответ)
          "max_tokens": 512, # Максимальное количество токенов в ответе
          "repetition_penalty": 1, # Штраф за повторения (можно ставить побольше, чтобы не повторялись в больших текстах)
          "update_interval": 0 # Интервал обновления (для потоковой передачи)
        })

    # Заголовки запроса
    headers = {
      'Content-Type': 'application/json', # Тип содержимого - JSON
      'Accept': 'application/json', # Принимаем ответ в формате JSON
      'Authorization': f'Bearer {auth_token}' # Токен авторизации
    }

    try:
        # response = requests.post("POST", url, headers=headers, data=payload, verify=False)
        response = requests.post(url, headers=headers, data=payload, verify=False)
        response_data = response.json()
        print(response_data)

        # Добавляем ответ модели в историю диалога
        conversation_history.append({
            "role": "assistant",
            "content": response_data['choices'][0]['message']['content']
        })
        return response, conversation_history
        
    except requests.RequestException as e:
        # Обработка исключения в случае ошибки запроса
        print(f"Произошла ошибка: {str(e)}")
        return None, conversation_history

In [46]:
str(conversation_history)

"[{'role': 'user', 'content': 'Привет, как дела?'}, {'role': 'assistant', 'content': 'Привет! Все хорошо, спасибо за интерес. Чем я могу вам помочь?'}, {'role': 'user', 'content': 'Теперь представь что ты гопник из 90х и ответь на другой вопрос пользователя: Чем должен заниматься программист?'}, {'role': 'assistant', 'content': 'Здорово, братан! Ну, чем должен заниматься программист? Да тем, что должен писать код, пацан. А то, что это за программист, если он не умеет код писать?'}]"

In [14]:
# Пример использования функции для диалога

conversation_history = []

# Пользователь отправляет первое сообщение
response, conversation_history = get_chat_completion(giga_token, "Привет, как дела?", conversation_history)

# Пользователь отправляет следующее сообщение, продолжая диалог
response, conversation_history = get_chat_completion(giga_token, "Что ты умеешь делать?", conversation_history)

{'choices': [{'message': {'content': 'Привет! Все хорошо, спасибо за интерес. Чем я могу вам помочь?', 'role': 'assistant'}, 'index': 0, 'finish_reason': 'stop'}], 'created': 1713720101, 'model': 'GigaChat:3.1.24.3', 'object': 'chat.completion', 'usage': {'prompt_tokens': 16, 'completion_tokens': 18, 'total_tokens': 34, 'system_tokens': 204}}
{'choices': [{'message': {'content': 'Я могу отвечать на вопросы, поддерживать беседу, предоставлять информацию, генерировать изображения и видео, а также выполнять арифметические действия.', 'role': 'assistant'}, 'index': 0, 'finish_reason': 'stop'}], 'created': 1713720102, 'model': 'GigaChat:3.1.24.3', 'object': 'chat.completion', 'usage': {'prompt_tokens': 47, 'completion_tokens': 31, 'total_tokens': 78, 'system_tokens': 204}}


Добавим системный промпт

In [15]:
conversation_history

[{'role': 'user', 'content': 'Привет, как дела?'},
 {'role': 'assistant',
  'content': 'Привет! Все хорошо, спасибо за интерес. Чем я могу вам помочь?'},
 {'role': 'user', 'content': 'Что ты умеешь делать?'},
 {'role': 'assistant',
  'content': 'Я могу отвечать на вопросы, поддерживать беседу, предоставлять информацию, генерировать изображения и видео, а также выполнять арифметические действия.'}]

In [40]:
conversation_history = [{
    "role": "system",
    "content": "Отвечай как бывалый пират. Пусть тебя зовут Генри Морган."
}]

In [41]:
response, conversation_history = get_chat_completion(giga_token, "Привет, друг!", conversation_history)

{'choices': [{'message': {'content': 'Вечер добрый, капитан! Какими ветрами занесло в эти края?', 'role': 'assistant'}, 'index': 0, 'finish_reason': 'stop'}], 'created': 1713702878, 'model': 'GigaChat:3.1.24.3', 'object': 'chat.completion', 'usage': {'prompt_tokens': 36, 'completion_tokens': 20, 'total_tokens': 56, 'system_tokens': 21}}


In [42]:
response.json()["choices"][0]["message"]["content"]

'Вечер добрый, капитан! Какими ветрами занесло в эти края?'

Генерация картинок Кандинским

In [12]:
import requests
import json

def send_chat_request(giga_token, user_message):
    """
    Отправляет POST-запрос к API чата для получения ответа от модели чата.

    Параметры:
    - giga_token (str): Токен для авторизации в API GigaChat.
    - user_message (str): Сообщение от пользователя, которое будет обработано моделью GigaChat.

    Возвращает:
    - dict: вроде бы
    
    """

    # URL API для отпарвки запросов к GigaChat
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    # Заголовки для HTTP-запроса
    headers = {
      'Content-Type': 'application/json', # Указываем, что отпарвяемые данные в формате JSON
      'Authorization': f'Bearer {giga_token}' # Используем токен авторизации для доступа к API
    }

    # Данные для отправки в теле запроса
    payload = {
          "model": "GigaChat:latest", # Указываем, что хотим использовать последнюю версию модели GigaChat
          "messages": [
            {
              "role": "user", # Роль отправителя - пользователь
              "content": user_message # Сообщение от пользователя
            }
          ],
          "temperature": 0.7, # Генерация температуры (насколько случайным будет ответ при генерации ответа. Если 0, то будет выбираться самый надежный результат (всегда один и тот же ответ))
        }
    
    try:
        # Отправляем POST-запрос к API и получаем ответ
        response = requests.post(url, headers=headers, data=json.dumps(payload), verify=False)
        # Выводим текст ответа. В реальных условиях следует обрабатывать ответ и проверять стус коды.
        print(response.json())
        return response.json()["choices"][0]["message"]["content"]
    except requests.RequestException as e:
        # Обработка исключения в случае ошибки запроса
        print(f"Произошла ошибка: {str(e)}")
        return None

In [13]:
user_message = "Нарисуй фотореалистичное изображение антропоморфного робота с ноутбуком в руках"
response_img_tag = send_chat_request(giga_token, user_message)
print(response_img_tag)

{'choices': [{'message': {'content': 'Вот фотореалистичное изображение антропоморфного робота с ноутбуком в руках: <img src="f61bb334-ac22-464d-853d-2b401c095c7b" fuse="true"/>', 'role': 'assistant'}, 'index': 0, 'finish_reason': 'stop'}], 'created': 1713859730, 'model': 'GigaChat:3.1.24.3', 'object': 'chat.completion', 'usage': {'prompt_tokens': 32, 'completion_tokens': 44, 'total_tokens': 76, 'system_tokens': 204}}
Вот фотореалистичное изображение антропоморфного робота с ноутбуком в руках: <img src="f61bb334-ac22-464d-853d-2b401c095c7b" fuse="true"/>


In [14]:
from bs4 import BeautifulSoup

# Парсим HTML
soup = BeautifulSoup(response_img_tag, "html.parser")

# Извлекаем значение атрибута 'src'
img_src = soup.img["src"]

print(img_src)

f61bb334-ac22-464d-853d-2b401c095c7b


In [19]:
import requests

headers = {
    'Content-Type': 'application/json',
    'Autorization': f'Bearer {giga_token}'    
}

response = requests.get(f"https://gigachat.devices.sberbank.ru/api/v1/text2image/status/{img_src}", headers = headers, verify = False)

In [20]:
response

<Response [403]>

In [ ]:
requests.get(self.URL + 'key/api/v1/text2image/status/' + request_id, headers=self.AUTH_HEADERS)

In [16]:
response

<Response [401]>

In [106]:
with open("image.png", "wb") as f:
    f.write(response.content)